In [1]:
import requests
import nba_api
import pandas as pd
import numpy as np
from datetime import date
from nba_api.stats.endpoints import (playbyplayv2,playbyplay, leaguehustlestatsplayer,
                                     leaguedashptstats,leaguestandings,
                                     playerdashptreb,leaguedashplayerbiostats)
import time

In [2]:
pbp2022=pd.read_csv(filepath_or_buffer="Data/2021-22_pbp.csv")

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)
Most charges drawn per 36 minutes (minimum 70% of games played), credit to morron88 for the idea to separate charges & loose balls in 2020

In [3]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season="2021-22",season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']]
compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="PerGame",season="2021-22",
    season_type_all_star="Regular Season").hustle_stats_player.get_data_frame()

In [4]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on='TEAM_ID',right_on='TeamID')
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [5]:
hustle_70percent_gp.nlargest(10,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','CHARGES_DRAWN']]

,PLAYER_NAME,CHARGES_DRAWN
349,Kevin Love,0.560000
429,Moritz Wagner,0.521053
197,Garrison Mathews,0.520152
466,Patrick Beverley,0.467717
364,Kyle Lowry,0.424779
156,Dewayne Dedmon,0.407547
148,Derrick White,0.405461
29,Austin Reaves,0.387931
93,Cody Martin,0.369582
413,Max Strus,0.339914


# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 70% of games played

In [6]:
hustle_70percent_gp.nlargest(10,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,LOOSE_BALLS_RECOVERED
297,John Konchar,1.367598
215,Hamidou Diallo,1.364384
491,Robert Williams III,1.350000
264,Jarred Vanderbilt,1.289764
139,Delon Wright,1.219048
466,Patrick Beverley,1.218898
367,LaMelo Ball,1.203715
93,Cody Martin,1.190875
106,Dalano Banton,1.188991
233,Ish Smith,1.186127


# The Plexiglass Award

most deflections per 36 minutes, minimum 70% of games played

In [7]:
hustle_70percent_gp.nlargest(10,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','DEFLECTIONS']]

,PLAYER_NAME,DEFLECTIONS
408,Matisse Thybulle,5.237647
200,Gary Payton II,5.195455
129,De'Anthony Melton,4.472247
304,Jordan McLaughlin,4.394483
138,Dejounte Murray,4.127586
490,Robert Covington,4.101099
19,Andre Drummond,3.910660
183,Facundo Campazzo,3.896703
264,Jarred Vanderbilt,3.855118
220,Herbert Jones,3.756522


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes, minimum 70% of games played

In [8]:
hustle_70percent_gp.nlargest(10,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','SCREEN_AST_PTS']]

,PLAYER_NAME,SCREEN_AST_PTS
499,Rudy Gobert,17.181308
528,Steven Adams,16.480608
249,Jakob Poeltl,14.933793
352,Kevon Looney,14.519431
217,Hassan Whiteside,14.319553
136,Deandre Ayton,14.155932
235,Ivica Zubac,14.075410
165,Drew Eubanks,14.029714
159,Domantas Sabonis,13.520930
111,Daniel Gafford,13.325373


# The No Fly Zone Award (presented by Dikembe Mutumbo)*

most blocked dunks as the blocking player

In [9]:
misses=pbp2022[pbp2022.HOMEDESCRIPTION.str.startswith('MISS',na=False)| 
           pbp2022.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
missed_dunks=misses[misses.HOMEDESCRIPTION.str.contains('Dunk',na=False)|
                   misses.VISITORDESCRIPTION.str.contains('Dunk',na=False)]
blocked_dunks=missed_dunks[missed_dunks.HOMEDESCRIPTION.str.contains('BLOCK',na=False)|
                   missed_dunks.VISITORDESCRIPTION.str.contains('BLOCK',na=False)]
#as the blocker
blocked_dunks.groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
105,Jaren Jackson Jr.,13
44,Daniel Gafford,11
78,Giannis Antetokounmpo,11
10,Andre Drummond,10
107,Jarrett Allen,9
143,Kristaps Porzingis,9
36,Clint Capela,8
47,Darius Bazley,8
70,Evan Mobley,8
95,JaVale McGee,8


# The Rejected for Boarding Award (sponsored by United Airlines)*

most blocked dunks as the dunking player (credit to Legdrop\_soup for the idea and asw7412 for the sponsor)

In [10]:
#as the blockee
blocked_dunks.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
148,Kenyon Martin Jr.,11
204,Precious Achiuwa,11
50,Darius Bazley,10
102,Ivica Zubac,10
48,Daniel Gafford,9
214,Rudy Gobert,9
143,Karl-Anthony Towns,7
192,OG Anunoby,7
227,Terance Mann,7
15,Bam Adebayo,6


# The “Oops, I Dunked It Again” Award (sponsored by Britney Spears, presented by Gary Payton & Shawn Kemp)*

Most prolific alley-oop duo (credit to lactardenthusiast for the idea)

In [11]:
#get alley oops
alley_oops=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Alley',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Alley',na=False)]
#remove missed alley oop attempts
made_alley_oops=alley_oops[~alley_oops.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~alley_oops.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
#remove alley oops that have missing 2nd player (self-alley oop or perhaps missing data?)
made_alley_oops_complete=made_alley_oops[~made_alley_oops.PLAYER2_NAME.isna()].copy()
made_alley_oops_complete['p1']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].min(axis=1)
made_alley_oops_complete['p2']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].max(axis=1)
made_alley_oops_complete.groupby(['p1','p2']).size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,p1,p2,count
277,Clint Capela,Trae Young,92
447,Dwight Powell,Luka Doncic,64
809,Mike Conley,Rudy Gobert,51
794,Marcus Smart,Robert Williams III,41
331,Darius Garland,Jarrett Allen,36
681,John Collins,Trae Young,31
259,Chris Paul,Deandre Ayton,27
329,Darius Garland,Evan Mobley,27
540,Immanuel Quickley,Obi Toppin,22
819,Mitchell Robinson,RJ Barrett,22


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played)

In [12]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').copy()
#traditional defensive stats approximate by tracking
#(deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
per_36_percent_ranks=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS_2PT','CONTESTED_SHOTS_3PT', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks["sum"]=per_36_percent_ranks.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks,how='left',left_index=True,right_index=True)

In [13]:
hustle_50percent_gp_ranks.to_csv(path_or_buf="Output Data/Hustle Ranks.csv")

In [14]:
hustle_50percent_gp_ranks.nsmallest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
537,Terrence Ross,0.150997,0.099715,0.190883,0.287749,0.119658,0.082621,0.931624
591,Will Barton,0.150997,0.210826,0.287749,0.219373,0.008547,0.105413,0.982906
178,Eric Gordon,0.448718,0.074074,0.005698,0.108262,0.282051,0.102564,1.021368
291,Joe Ingles,0.150997,0.034188,0.247863,0.196581,0.487179,0.022792,1.139601
57,Bryn Forbes,0.150997,0.014245,0.011396,0.071225,0.515670,0.396011,1.159544
101,D.J. Augustin,0.618234,0.005698,0.028490,0.059829,0.113960,0.339031,1.165242
484,Reggie Bullock,0.150997,0.176638,0.022792,0.304843,0.236467,0.336182,1.227920
133,DeMar DeRozan,0.301994,0.076923,0.014245,0.262108,0.444444,0.162393,1.262108
22,Anfernee Simons,0.444444,0.105413,0.481481,0.031339,0.011396,0.256410,1.330484
0,Aaron Gordon,0.339031,0.544160,0.111111,0.056980,0.245014,0.179487,1.475783


In [15]:
hustle_50percent_gp_ranks[hustle_50percent_gp_ranks['PLAYER_NAME'].isin(['Patrick Beverley','Russell Westbrook'])].filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
466,Patrick Beverley,0.988604,0.190883,0.065527,0.928775,0.048433,0.877493,3.099715
501,Russell Westbrook,0.826211,0.068376,0.113960,0.467236,0.293447,0.729345,2.498575


# The "Mr. Fantastic" Award (presented by Shane Battier, the No-Stats All-Star)*

Highest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played) (credit to memeticengineering for the idea)

In [16]:
hustle_50percent_gp_ranks.nlargest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
540,Thaddeus Young,0.985755,0.752137,0.977208,0.951567,0.846154,0.968661,5.481481
164,Draymond Green,0.888889,0.792023,0.923077,0.886040,0.923077,0.863248,5.276353
134,DeMarcus Cousins,0.968661,0.849003,0.769231,0.940171,0.900285,0.772080,5.199430
314,Josh Okogie,0.943020,0.578348,0.908832,0.965812,0.652422,0.707977,4.756410
17,Alperen Sengun,0.925926,0.914530,0.706553,0.851852,0.911681,0.408832,4.719373
440,Naz Reid,0.757835,0.877493,0.874644,0.641026,0.934473,0.529915,4.615385
37,Blake Griffin,1.000000,0.754986,0.800570,0.524217,0.851852,0.569801,4.501425
234,Ish Wainright,0.960114,0.259259,0.739316,0.834758,0.803419,0.886040,4.482906
220,Herbert Jones,0.806268,0.569801,0.925926,0.960114,0.643875,0.572650,4.478632
429,Moritz Wagner,0.994302,0.763533,0.774929,0.518519,0.977208,0.450142,4.478632


# The Bowling Ball Award (sponsored by [Pete Weber](https://www.youtube.com/watch?v=gKQOXYB2cd8), presented by Glen "Big Baby" Davis)*

most charges committed (credit to Kdog122025 for the idea)

In [17]:
charges=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Charge',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Charge',na=False)]

charges.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
122,Giannis Antetokounmpo,23
262,Pascal Siakam,16
191,Jusuf Nurkic,14
211,Kyle Kuzma,14
22,Bam Adebayo,13
303,Tobias Harris,13
187,Julius Randle,12
0,Aaron Gordon,11
156,Jaren Jackson Jr.,11
164,Jayson Tatum,11


# "The Good Ol' Hockey Game, is the Best Game You Can Name" Award (presented by Dominik Hasek)*

most goaltends committed (credit to Kdog122025 for the idea)

In [18]:
goaltends=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Goaltending',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Goaltending',na=False)]
goaltends.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
39,Clint Capela,20
94,JaVale McGee,16
8,Andre Drummond,13
0,Aaron Gordon,12
161,Mo Bamba,11
166,Myles Turner,11
79,Greg Brown III,10
160,Mitchell Robinson,10
192,Richaun Holmes,10
194,Robert Williams III,10


# The Anti-Verticality Award (presented by Roy Hibbert)*

most 3-point shooting fouls committed (credit to watchingsongsDL, kingcobweb & An-Indian-In-The-NBA for the idea)

In [19]:
third_free_throws=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)|
                          pbp2022['VISITORDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)]

shooting_fouls=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('S.FOUL',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('S.FOUL',na=False)]

fouls_on_missed_threes=shooting_fouls.merge(right=third_free_throws,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

three_pointers=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('3PT',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('3PT',na=False)]

made_threes=three_pointers[~three_pointers.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~three_pointers.VISITORDESCRIPTION.str.startswith('MISS',na=False)]

fouls_on_made_threes=shooting_fouls.merge(right=made_threes,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

fouls_on_threes=fouls_on_missed_threes.append(fouls_on_made_threes)
fouls_on_threes.groupby('PLAYER1_NAME_x').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME_x,count
96,Desmond Bane,10
25,Bam Adebayo,7
38,Buddy Hield,7
84,De'Anthony Melton,7
125,Garrett Temple,7
199,Josh Richardson,7
247,Malik Monk,7
255,Matisse Thybulle,7
262,Miles Bridges,7
329,Talen Horton-Tucker,7


# The “Fine, I’ll Do It Myself” Award (sponsored by Thanos, presented by Allen Iverson)

Highest percentage of unassisted field goals, minimum 50% of games played (https://www.nba.com/stats/players/scoring/?sort=GP&dir=-1)

1. Luka Doncic (85.8%)
2. Chris Paul (84.8%)
3. Shai Gilgeous-Alexander (83.2%)
4. Trae Young (83.1%)
5. James Harden (78.6%)

# The “You Gotta Feed Me” Award (presented by Joey Chestnut & Marcin Gortat)

Highest percentage of assisted field goals, minimum 50% of games played

1. Davis Bertans (96%)
2. Wayne Ellington (93.9%)
3. Duncan Robinson (93.4%)
4. Ben McLemore (93.1%)
5. Mike Muscala & Maxi Kleber (92.9%)

# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award

Lowest contested rebound percentage, minimum 50% of games played

In [20]:
rebounding=playerdashptreb.PlayerDashPtReb(team_id=0,player_id=0).overall_rebounding.get_data_frame()

games_percentages=hustle_w_gp_qualify.copy()[['PLAYER_ID','PLAYER_NAME','G_PERCENT']]

reb_w_gp_qualify=rebounding.merge(games_percentages,left_on='PLAYER_ID',right_on='PLAYER_ID').query('G_PERCENT >= 0.5')

reb_w_gp_qualify.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
29,D.J. Augustin,0.061
87,Tyus Jones,0.076
5,Isaiah Joe,0.096
149,Trae Young,0.097
515,Aaron Holiday,0.103
277,Dennis Schroder,0.106
183,Bones Hyland,0.109
428,Armoni Brooks,0.109
333,Immanuel Quickley,0.111
85,Raul Neto,0.114


In [21]:
reb_w_gp_qualify[reb_w_gp_qualify['PLAYER_NAME']=='Carmelo Anthony'][['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
340,Carmelo Anthony,0.311


alternatively: restricting to players > 6 foot 6 inches in height

In [22]:
player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats().league_dash_player_bio_stats.get_data_frame()

above_66=player_bio.query('PLAYER_HEIGHT_INCHES > 6*12+6')

above_66.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
45,Furkan Korkmaz,0.129
40,Duncan Robinson,0.143
162,Ziaire Williams,0.152
4,Amir Coffey,0.161
14,Cam Reddish,0.162
82,Jordan Nwora,0.171
77,Joe Ingles,0.174
17,Cedi Osman,0.182
151,Tomas Satoransky,0.185
105,Marcus Morris Sr.,0.191


# The "Where There's a Will, There's a Way" Award (presented by Dennis Rodman)

Highest contested rebound percentage, minimum 50% of games played

In [23]:
reb_w_gp_qualify.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
447,Mitchell Robinson,0.605
27,Jakob Poeltl,0.576
141,Jock Landale,0.565
475,Ivica Zubac,0.523
454,Steven Adams,0.521
394,Jericho Sims,0.510
165,Goga Bitadze,0.506
170,Tristan Thompson,0.495
412,JaVale McGee,0.492
529,Robert Williams III,0.491


alternatively: restricting to players < 6 foot 7 inches in height

In [24]:
below_67=player_bio.query('PLAYER_HEIGHT_INCHES < 6*12+7')
below_67.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
169,Thanasis Antetokounmpo,0.438
85,Ish Wainright,0.420
116,Kenrich Williams,0.391
88,Jae'Sean Tate,0.377
161,Stanley Johnson,0.374
107,Josh Okogie,0.358
118,Kenyon Martin Jr.,0.357
154,Romeo Langford,0.355
74,Gary Payton II,0.349
94,Javonte Green,0.349


# The "David vs Goliath" Award (presented by Dwyane Wade)*

most shots blocked with at least a 5-inch difference in height

In [25]:
blocks=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('BLOCK',na=False)|
                          pbp2022['VISITORDESCRIPTION'].str.contains('BLOCK',na=False)]

blocking_player_heights=blocks.merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER1_ID',right_on='PLAYER_ID').merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER3_ID',right_on='PLAYER_ID')
blocking_player_heights["height_difference"]=blocking_player_heights['PLAYER_HEIGHT_INCHES_x']-blocking_player_heights['PLAYER_HEIGHT_INCHES_y']
blocking_player_heights.query('height_difference >= 5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
196,Patrick Beverley,28
88,Fred VanVleet,25
85,Facundo Campazzo,18
234,Terry Rozier,16
67,Derrick White,15
112,Ish Smith,15
37,Chris Paul,14
57,De'Anthony Melton,13
77,Draymond Green,13
11,Anthony Edwards,12


# The Rotation Awards

**value depth over one solitary star. used basketball-reference's play-by-play position percentages as baseline for position groupings, minimum 250 MP & 12 MP/G**

# The Best Guard Rotation Award (sponsored by Buckingham Palace)

East:
1. Heat (Lowry, Herro, Vincent)
3. Sixers (Maxey, *Harden*, Seth, Shake)
4. Bulls (Lonzo, Caruso, Coby, Ayo)
6. Cavaliers (Garland, Rubio, Sexton, LeVert, Rondo, Brandon Goodwin)
8. Nets (Harden, *Seth*, Patty Mills, World B Flat, Jevon Carter, Dragic, David Duke Jr.)
5. Boston (Smart, Fast PP, Schroder, Derrick White)
12. Knicks (Quickley, Quentin Grimes, Burks, Rose, Kemba)
13. Pacers (Duarte, Brogdon, Haliburton, *Hield*, Duane Washington, TJ McConnell, Keifer Sykes, Brad Wanamaker)
14. Pistons (Cade, Cory Jo, Killian, Frank Jackson, Saben Lee)

West:

1. Suns (CP3, *Book*, Cam Payne, Aaron Holiday)
3. Warriors (Steph, Poole, GPII)
5. Mavericks (Luka, Brunson, Dinwiddie, Frankie Smokes)
4. Jazz (Spida, Conley, Trent Forrest, *Clarkson*)
2. Grizzlies (Ja, Melton, Tyus Jones, Konchar)
11. Spurs (Dejounte, Derrick White, Tre Jones, Bryn Forbes)
13. Kings (Fox, Hali, Davion Mitchell, *Hield*)
12. Trail Blazers (Dame, Simons, *McCollum*, *Norm Powell*, DSJ, Brandon Williams, Kris Dunn)
14. Thunder (SGA, *Giddey*, Tre Mann, Ty Jerome, Theo Maledon)

# The Best Wing Rotation Award (co-sponsored by Lou Williams and Magic City)

East:
1. Boston (Jay^2, Josh Richardson, Langford)
4. Bulls (LaVine, Deebo, Javonte Green, TBJ, DJJ)
1. Heat (Butler, Strus, Sheen, Caleb Martin, Oladipo)
2. Bucks (Middleton, Connaughton, Jordan Nwora, Hood, Wesley Matthews, Ted Cruz)
9. Hornets (Miles Bridges, Hayward, Jalen McDaniels, Oubre, Cody Martin)
10. Hawks (Huerter, Bogdan^2, Hunter, Reddish, TLC)

West:

1. Suns (*Cam Johnson*, Bridges, Waluigi, Crowder, Torrey Craig)
2. Grizzlies (Bane, Ziaire Williams, Dillon Brooks, *Kyle Anderson*)
10. Pelicans (Herb Jones, Brandon Ingram, Josh Hart, NAW, Trey Murphy, Naji Marshall, Tomas Satoransky, Tony Snell)
5. Mavericks (DFS, Bullock, THJ, Josh Green, Sterling Brown)
8. Clippers (PG13, Terance Mann, Marcus Morris, Covington, Amir Coffey, Brandon Boston)
3. Warriors (Wiggins, JTA, Iggy, Klay, Damion Lee)
11. Spurs (Big Body, Keita Bates-Diop, Vassell, Lonnie Walker, Primo, Josh Richardson)
14. Thunder (Aaron Wiggins, DORT, Bazley, Kenrich, Lindy Waters, Vit Krejci)
15. Rockets (Jae'Sean Tate, Kenyon Martin Jr, Nwaba, Eric Gordon, Garry Bird, Josh Christopher)

# The Best Big Rotation Award (jointly sponsored by Tom Hanks, Cadbury and Sex and the City)

East:

1. Cavaliers (Allen, Mobley, Markkanen, *Love*)
5. Boston (Horford, Timelord, Theis, *Grant Williams*)
2. Bucks (Giannis, Bobby Portis, Ibaka, Cousins, Brook Lopez, Semi Ojeleye)
3. Sixers (Embiid, Tobi, Drummond)
1. Heat (Bam, PJ Tucker, Dedmon, Yurt7, Markieff)
7. Raptors (Siakam, ~~Pascal~~, Achiuwa, Khem Birch, *Boucher*, *Thad Young*)
8. Nets (KD, LMA, Blake Griffin, Claxton, Drummond, Day'Ron Sharpe)
10. Hawks (Capela, John Collins, Gallinari, Okongwu)
13. Pacers (Sabonis, Myles Turner, Goga, IJax, Jalen Smith)
11. Wizards (Kuzma, Montrezl, Gafford, Thomas Bryant, Porzingis, Bertans)
14. Magic (WCJ, Bamba, Mo Wagner, Robin Lopez)

West:

1. Grizzlies (JJJ, Steven Adams, *Brandon Clarke*, *Xavier Tillman*, Killian Tillie)
3. Warriors (Dray, Looney, OPJ, Bjelica, Kuminga)
4. Jazz (Gobert, Whiteside, Rudy Gay, Paschall)
1. Suns (Ayton, JaVale, Jalen Smith, Bismack, Kaminsky)
5. Mavericks (Dwight Powell, Kleber, Porzingis, Bertans)
7. Wolves (KAT, Vanderbilt, Naz Reid)
6. Nuggets (Jokic, Jeff Green, Nnaji, DeMarcus Cousins, *JaMychal*)
10. Pelicans (Valanciunas, Jaxson Hayes, Willy Hernangomez)
9. Lakers (Day-to-Dayvis, Carmelo, Dwight, Stanley Johnson, DeAndre Jordan)
12. Trail Blazers (Nurk, Covington, Larry Nance Jr, Trendon Watford, Eubanks, Cody Zeller)
15. Rockets (Christian Wood, Alperen, Theis)